# TP : Introduction à la compression JPEG 2000

**Niveau :** Première NSI

**Objectifs :**
- Comprendre le principe de base de la compression d'images
- Manipuler des calculs de moyennes et différences
- Découvrir la transformation de Haar sur 2 puis 4 points

## Introduction : Pourquoi compresser une image ?

### Le problème

Une image de **816 × 608 pixels** en couleur nécessite :
- 3 octets par pixel (rouge, vert, bleu)
- Soit : 816 × 608 × 3 = **1 488 384 octets** ≈ **1,49 Mo**

Avec la compression JPEG, cette même photo ne pèse que **237 Ko**, soit environ **6,28 fois moins** !

### L'idée principale

Dans une région uniforme (ciel bleu, mur blanc), les pixels voisins ont des valeurs très proches.

Au lieu de stocker chaque pixel individuellement, on peut stocker :
- Leur **moyenne** (qui représente la région)
- Leurs **différences** (qui sont souvent très petites)

Les petites différences peuvent être arrondies à 0 pour **compresser** l'information !

C'est le principe de la **transformation en ondelettes** utilisée par JPEG 2000.

### Simplification pour ce TP

Pour simplifier, nous allons travailler sur une image **en niveaux de gris** (1 seul canal).

En pratique, pour compresser une image couleur :
- On convertit l'image RGB en niveaux de gris, OU
- On applique la transformation séparément sur chaque canal (Rouge, Vert, Bleu)

Dans ce TP, nous nous concentrons sur le principe de base avec des niveaux de gris.

## Importation des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

## Partie 1 : Transformation sur 2 points

### Le principe

Au lieu de stocker deux nombres `a` et `b`, on peut stocker :
- Leur **moyenne** : $x = \frac{a+b}{2}$
- La **moitié de leur différence** : $y = \frac{b-a}{2}$

On peut retrouver `a` et `b` avec :
- $b = x + y$
- $a = x - y$

**Intérêt** : Si `a` et `b` sont proches, alors `y` sera très petit et pourra être arrondi à 0 pour économiser de l'espace !

### 📝 Exercice 1 : Transformation de deux pixels

Voici les tons de gris de deux pixels voisins : `a = 25` et `b = 23`.

Calculez leur moyenne `x` et leur demi-différence `y`.

In [ ]:
# Deux pixels voisins
a = 25
b = 23

# TODO: Calculer la moyenne x
x = 

# TODO: Calculer la demi-différence y
y = 

print(f"Pixels originaux : a={a}, b={b}")
print(f"Transformés : moyenne={x}, demi-différence={y}")

### 📝 Exercice 2 : Vérification de la reconstruction

Vérifiez qu'on peut retrouver `a` et `b` à partir de `x` et `y`.

In [ ]:
# TODO: Reconstruire a et b à partir de x et y
a_reconstruit = 
b_reconstruit = 

print(f"Reconstruction : a={a_reconstruit}, b={b_reconstruit}")
print(f"Vérification : a original={a}, b original={b}")

### 🔍 Observation

Dans cet exemple :
- La moyenne est `x = 24`
- La demi-différence est `y = -1` (très petit !)

Si on arrondissait `y` à 0, on ne perdrait presque aucune information, mais on pourrait stocker l'information plus efficacement.

### 📊 Visualisation graphique

In [ ]:
# Visualisation graphique de la transformation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Avant la transformation
axes[0].bar(['a', 'b'], [a, b], color=['#3498db', '#e74c3c'])
axes[0].set_ylabel('Valeur du pixel')
axes[0].set_title('AVANT : Pixels originaux')
axes[0].set_ylim(0, 30)
axes[0].grid(axis='y', alpha=0.3)

# Après la transformation
axes[1].bar(['x (moyenne)', 'y (demi-diff)'], [x, y], color=['#2ecc71', '#9b59b6'])
axes[1].set_ylabel('Valeur')
axes[1].set_title('APRÈS : Transformation de Haar')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f">>> On voit que la demi-différence y={y} est très petite !")
print(f"    Elle pourrait être arrondie à 0 pour compresser.")

## Partie 2 : Transformation sur 4 points (carré 2×2)

### Le principe en 2 dimensions

Pour un carré 2×2 de pixels :

```
a  b
c  d
```

La **transformation de Haar** calcule 4 valeurs :

1. **Rouge (moyenne globale)** : $\frac{a+b+c+d}{4}$ 
   - Représente la couleur globale du carré

2. **Bleu (différences verticales)** : $\frac{(b-a)+(d-c)}{4}$ 
   - Mesure les changements de gauche à droite

3. **Vert (différences horizontales)** : $\frac{(c-a)+(d-b)}{4}$ 
   - Mesure les changements de haut en bas

4. **Violet (différences obliques)** : $\frac{(b+c)-(a+d)}{4}$ 
   - Mesure les changements en diagonale

### 📝 Exercice 3 : Implémenter la transformation de Haar 2×2

In [ ]:
def transformation_haar_2x2(carre):
    """
    Applique la transformation de Haar sur un carré 2x2.
    
    Paramètres:
        carre: tableau numpy 2x2
    
    Retourne:
        tableau 2x2 transformé avec:
        [rouge, bleu]
        [vert, violet]
    """
    a, b = carre[0, 0], carre[0, 1]
    c, d = carre[1, 0], carre[1, 1]
    
    # TODO: Calculer les 4 coefficients
    rouge =   # Moyenne globale
    bleu =    # Différences verticales
    vert =    # Différences horizontales
    violet =  # Différences obliques
    
    return np.array([[rouge, bleu], [vert, violet]])

### Test de la fonction

Testons avec un petit carré 2×2 extrait d'une photo :

In [ ]:
# Exemple : un carré 2x2 extrait d'une vraie photo
carre_test = np.array([[134, 224],
                       [137, 162]], dtype=float)

resultat = transformation_haar_2x2(carre_test)

print("Carré original :")
print(carre_test)
print("\nCarré transformé :")
print(resultat)
print("\nDétail :")
print(f"Rouge (moyenne globale) : {resultat[0,0]:.2f}")
print(f"Bleu (vertical) : {resultat[0,1]:.2f}")
print(f"Vert (horizontal) : {resultat[1,0]:.2f}")
print(f"Violet (oblique) : {resultat[1,1]:.2f}")

### 🔍 Analyse des résultats

Résultats attendus :
- **Rouge ≈ 164** : c'est la couleur moyenne du carré
- **Bleu ≈ 26** : différence importante (changement vertical)
- **Vert ≈ -16** : différence moyenne (changement horizontal)
- **Violet ≈ 11** : petite différence diagonale

**Interprétation** : 
- La valeur rouge contient l'information principale
- Les trois autres valeurs (bleu, vert, violet) représentent les détails
- Dans une zone uniforme, ces trois valeurs seraient proches de 0 !

### 📊 Visualisation graphique du carré 2×2

In [ ]:
# Visualisation du carré 2×2 avec PIL
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# === AVANT : Carré original ===
# Créer une image PIL en niveaux de gris
img_original = Image.fromarray(carre_test.astype('uint8'), mode='L')
# Agrandir pour mieux voir (chaque pixel devient 100x100)
img_original_grande = img_original.resize((200, 200), Image.NEAREST)

axes[0].imshow(img_original_grande, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('AVANT : Carré original 2×2', fontsize=14, fontweight='bold')
axes[0].set_xticks([50, 150])
axes[0].set_yticks([50, 150])
axes[0].set_xticklabels(['a,c', 'b,d'])
axes[0].set_yticklabels(['a,b', 'c,d'])
axes[0].axis('on')

# Ajouter les valeurs sur l'image
for i in range(2):
    for j in range(2):
        axes[0].text(j*100 + 50, i*100 + 50, f'{int(carre_test[i, j])}', 
                    ha='center', va='center', color='red', 
                    fontsize=24, fontweight='bold')

# === APRÈS : Carré transformé ===
# Créer une image colorée pour mieux visualiser les différents coefficients
img_transfo = Image.new('RGB', (200, 200))
draw = ImageDraw.Draw(img_transfo)

# Définir les couleurs pour chaque quadrant
couleurs = [
    [(200, 50, 50), (50, 50, 200)],   # Rouge, Bleu
    [(50, 200, 50), (150, 50, 150)]   # Vert, Violet
]

labels = [
    ['ROUGE\n(moyenne)', 'BLEU\n(vertical)'],
    ['VERT\n(horizontal)', 'VIOLET\n(oblique)']
]

# Dessiner les 4 quadrants colorés
for i in range(2):
    for j in range(2):
        x0, y0 = j * 100, i * 100
        draw.rectangle([x0, y0, x0 + 100, y0 + 100], fill=couleurs[i][j])

axes[1].imshow(img_transfo)
axes[1].set_title('APRÈS : Transformation de Haar', fontsize=14, fontweight='bold')
axes[1].set_xticks([50, 150])
axes[1].set_yticks([50, 150])
axes[1].set_xticklabels(['quadrant 1', 'quadrant 2'])
axes[1].set_yticklabels(['quadrant 1', 'quadrant 2'])
axes[1].axis('on')

# Ajouter les labels et valeurs
for i in range(2):
    for j in range(2):
        axes[1].text(j*100 + 50, i*100 + 50, 
                    f'{labels[i][j]}\n{resultat[i, j]:.1f}', 
                    ha='center', va='center', color='white', 
                    fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("Interprétation :")
print(f"  • ROUGE ({resultat[0,0]:.1f}) = valeur moyenne du carré")
print(f"  • BLEU ({resultat[0,1]:.1f}) = changements verticaux (gauche -> droite)")
print(f"  • VERT ({resultat[1,0]:.1f}) = changements horizontaux (haut -> bas)")
print(f"  • VIOLET ({resultat[1,1]:.1f}) = changements en diagonale")

### 📝 Exercice 4 : Test avec un carré uniforme

Testons avec un carré où tous les pixels ont la même valeur :

In [ ]:
# Carré uniforme (tous les pixels à 100)
carre_uniforme = np.array([[100, 100],
                           [100, 100]], dtype=float)

resultat_uniforme = transformation_haar_2x2(carre_uniforme)

print("Carré uniforme :")
print(carre_uniforme)
print("\nCarré transformé :")
print(resultat_uniforme)
print("\nQue remarquez-vous ?")

## Conclusion

### Ce que nous avons appris

1. **Principe de base** : On peut remplacer des valeurs par leur moyenne et leurs différences

2. **Intérêt pour la compression** :
   - Les moyennes contiennent l'information principale
   - Les différences sont souvent petites dans les zones uniformes
   - On peut arrondir les petites différences à 0 pour compresser

3. **Transformation de Haar** :
   - Sur 2 points : on obtient 1 moyenne + 1 différence
   - Sur 4 points (2×2) : on obtient 1 moyenne + 3 différences (verticale, horizontale, diagonale)

### Et après ?

Pour compresser une vraie image :
- On divise l'image en petits carrés 2×2
- On applique la transformation sur chaque carré
- On peut même réappliquer la transformation plusieurs fois (sur les moyennes)
- On arrondit à 0 les petites valeurs pour compresser

C'est le principe du **JPEG 2000**, qui utilise des ondelettes plus sophistiquées que celle de Haar !

## 🎯 Questions de réflexion

1. **Pourquoi la transformation en moyennes et différences est-elle utile pour la compression ?**

2. **Dans quel cas les différences (bleu, vert, violet) seront-elles proches de 0 ?**

3. **Quel type d'image se compresse le mieux : une photo d'un ciel bleu uniforme ou une photo d'une foule de personnes ? Pourquoi ?**

4. **À votre avis, peut-on reconstruire exactement l'image originale à partir de la transformation ? Pourquoi ?**

---

**Source :** D'après un article de Christiane Rousseau, Université de Montréal  
**Adapté pour 1ère NSI - Version simplifiée**